In [26]:
import json
import requests
import datetime
import pandas as pd
import geo_utils as gu
from north_sea.src.utils import check_countries

In [27]:
def get_nl_production(url, start_year):

    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    dfs = []
    for year in range(start_year, (datetime.date.today().year + 1)):
        params = {"product": "Gas", "production": "Produced","year": f"{year}"}
        r = requests.post(url, headers=headers, data = json.dumps(params))
        df = pd.DataFrame.from_records(r.json())
        dfs.append(df)
    df = pd.concat(dfs)

    return df

def get_production(countries, year=2003):
    
    with open('sources.json') as f:
        sources = json.load(f)
    
    nl_urls = []
    names = []

    for source in sources:
        if 'production' in source.get('tags'):
            if 'no' in source.get('countries'):
                no_url = source.get('url')
            elif 'nl' in source.get('countries'):
                nl_urls.append(source.get('url'))
                names.append(source.get('url')[52:])
            elif 'uk' in source.get('countries'):
                uk_url = source.get('url')

    for country in check_countries(countries):
        
        name = 'production'
        if country == 'no':
            df = pd.read_csv(no_url) 
        elif country == 'uk':
            pass # TODO: fix wfs import 
        gu.gdf_to_postgres(df, name , country, 'replace')
        elif country == 'nl':
            for name, url in zip(names, nl_urls):
                df = get_production(url, 2003)
                name = f'production_{name}'                
                gu.gdf_to_postgres(df, name , country, 'replace')
                  